In [75]:
import pandas as pd

training = pd.read_csv("new_training.csv")
test = pd.read_csv("new_test.csv")

In [76]:
training.head()

,sentence,difficulty,encoded_diff,sentence_sp,tokens,tokens_no_stop,token_count,nb_verbs,nb_nouns,nb_adj,nb_adv,words,tfidf_score_unigram,tfidf_score_bigram
0,Les coûts kilométriques réels peuvent diverger...,C1,4,Les coûts kilométriques réels peuvent diverger...,"['Les', 'coûts', 'kilométriques', 'réels', 'pe...","['coûts', 'kilométriques', 'réels', 'diverger'...",27,4,14,3,2,"['les', 'coûts', 'kilométriques', 'réels', 'pe...",27.043962,18.477813
1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1,0,"Le bleu, c'est ma couleur préférée mais je n'a...","['Le', 'bleu', ',', ""c'"", 'est', 'ma', 'couleu...","['bleu', ',', 'couleur', 'préférée', 'aime', '...",7,1,3,1,2,"['le', 'bleu,', ""c'est"", 'ma', 'couleur', 'pré...",6.295061,4.283070
2,Le test de niveau en français est sur le site ...,A1,0,Le test de niveau en français est sur le site ...,"['Le', 'test', 'de', 'niveau', 'en', 'français...","['test', 'niveau', 'français', 'site', 'Intern...",7,1,4,1,0,"['le', 'test', 'de', 'niveau', 'en', 'français...",35.379919,23.504444
3,Est-ce que ton mari est aussi de Boston?,A1,0,Est-ce que ton mari est aussi de Boston?,"['Est', '-ce', 'que', 'ton', 'mari', 'est', 'a...","['-ce', 'mari', 'Boston', '?']",4,0,2,0,1,"['est-ce', 'que', 'ton', 'mari', 'est', 'aussi...",6.326896,4.309433
4,"Dans les écoles de commerce, dans les couloirs...",B1,2,"Dans les écoles de commerce, dans les couloirs...","['Dans', 'les', 'écoles', 'de', 'commerce', ',...","['écoles', 'commerce', ',', 'couloirs', 'place...",24,4,10,2,1,"['dans', 'les', 'écoles', 'de', 'commerce,', '...",88.671934,60.137653


In [77]:
y = training['encoded_diff']
X = training[['token_count','nb_verbs', 'nb_nouns', 'nb_adj', 'nb_adv','tfidf_score_unigram', 'tfidf_score_bigram']]

In [78]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [79]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty=None, solver='lbfgs', max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

Best Parameters: {'n_estimators': 100}


In [80]:
# Accuracy on the test set
print('Accuracy of Logistic regression classifier on test set: {:.2f}'
     .format(model.score(X_test, y_test)))

# Accuracy on the training set
print('Accuracy of Logistic regression classifier on training set: {:.2f}'
     .format(model.score(X_train, y_train)))

Accuracy of Logistic regression classifier on test set: 0.34
Accuracy of Logistic regression classifier on training set: 0.99


In [81]:
X_to_predict = test[['token_count','nb_verbs', 'nb_nouns', 'nb_adj', 'nb_adv','tfidf_score_unigram', 'tfidf_score_bigram']]

In [82]:
test['difficulty encoded'] = model.predict(X_to_predict)

In [83]:
#function that cleans and return the final dataset to upload on kaggle
def prep_final_pred(df):
  number_to_level = {
    0: 'A1',
    1: 'A2',
    2: 'B1',
    3: 'B2',
    4: 'C1',
    5: 'C2'
}

  df['difficulty'] = df['difficulty encoded'].map(number_to_level)
  columns_to_keep = ['id', 'difficulty']
  final_df =  df[columns_to_keep]
  final_df.set_index('id', inplace=True)


  return final_df


In [84]:
final_test = prep_final_pred(test)

In [85]:
final_test.to_csv('submission.csv')

In [86]:
final_test.head()

,difficulty
id,
0,A2
1,B2
2,C2
3,B2
4,C2


In [87]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [88]:
! pip install kaggle

In [89]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [90]:
!cp /content/drive/MyDrive/Kaggle/kaggle.json ~/.kaggle/kaggle.json

In [91]:
! kaggle competitions submit -c detecting-french-texts-difficulty-level-2023 -f submission.csv -m "Sample submission"


100% 8.30k/8.30k [00:00<00:00, 32.8kB/s]
Successfully submitted to Detecting the difficulty level of French texts